In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import os
import sklearn
import time
import seaborn as sns
import pickle
import cv2
import shap

2025-11-14 10:21:20.692086: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-14 10:21:20.757722: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-14 10:21:22.366649: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/le-cuong/Desktop/BTL_Machine_Learning/.venv/lib/python3.12/site-packages/tqdm/auto.py

In [4]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Data Exploration
- We chose the chest-x-ray datasets from Kaggle: https://www.kaggle.com/datasets/habibmrad1983/chestxray8-dataset

#### Split dataset into train dataset and test dataset

In [10]:
# Read csv-file
df = pd.read_csv("data/train-all.csv")
valid_df = pd.read_csv("data/valid-all.csv")
print(f'There are {df.shape[0]} rows and {df.shape[1]} columns in the train data frame')
print(df.head())

print("\n")

print(f'There are {valid_df.shape[0]} rows and {valid_df.shape[1]} columns in the train data frame')
print(valid_df.head())

There are 99316 rows and 16 columns in the train data frame
              Image  Atelectasis  Cardiomegaly  Consolidation  Edema  \
0  00000001_000.png            0             1              0      0   
1  00000001_001.png            0             1              0      0   
2  00000001_002.png            0             1              0      0   
3  00000002_000.png            0             0              0      0   
4  00000004_000.png            0             0              0      0   

   Effusion  Emphysema  Fibrosis  Hernia  Infiltration  Mass  Nodule  \
0         0          0         0       0             0     0       0   
1         0          1         0       0             0     0       0   
2         1          0         0       0             0     0       0   
3         0          0         0       0             0     0       0   
4         0          0         0       0             0     1       1   

   PatientId  Pleural_Thickening  Pneumonia  Pneumothorax  
0          1  

In [14]:
from sklearn.model_selection import train_test_split

test_size = 0.2

train_df, test_df = train_test_split(
    df,
    test_size=test_size,
    random_state=42
    # stratify=df['Finding Labels']
)

print(f"Toàn bộ: {len(df)} ảnh")
print(f"Train:   {len(train_df)} ảnh ({len(train_df)/len(df):.1%})")
print(f"Test:    {len(test_df)} ảnh ({len(test_df)/len(df):.1%})")

train_df.to_csv('data/train.csv', index=False)
test_df.to_csv('data/test.csv', index=False)

print("\nĐã lưu thành công 2 files: train.csv, test.csv")

Toàn bộ: 99316 ảnh
Train:   79452 ảnh (80.0%)
Test:    19864 ảnh (20.0%)

Đã lưu thành công 2 files: train.csv, test.csv


#### Prevent Data Leakage

In [15]:
def check_for_leakage(df1, df2, patient_col):
    """
    Return True if there any patients are in both df1 and df2.

    Args:
        df1 (dataframe): dataframe describing first dataset
        df2 (dataframe): dataframe describing second dataset
        patient_col (str): string name of column with patient IDs
    
    Returns:
        leakage (bool): True if there is leakage, otherwise False
    """

    df1_patients_unique = set(df1[patient_col])
    df2_patients_unique = set(df2[patient_col])

    patients_in_both_groups = list(df1_patients_unique.intersection(df2_patients_unique))

    leakage = len(patients_in_both_groups) > 0

    return leakage